# Inventory Data Stage Load

**This notebook processes inventory data from source files and loads it into the staging table**

In [0]:
# Configuration
source_dir = "/Volumes/event-driven-catalog/default/incremental_load/inventory_data/source/"
archive_dir = "/Volumes/event-driven-catalog/default/incremental_load/inventory_data/archive/"
stage_table = "`event-driven-catalog`.default.inventory_stage"


print(f"Processing inventory data from: {source_dir}")
print(f"Staging table: {stage_table}")


Processing inventory data from: /Volumes/event-driven-catalog/default/incremental_load/inventory_data/source/
Staging table: `event-driven-catalog`.default.inventory_stage


In [0]:
# Import required libraries
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime
import json

# Define schema for inventory data
inventory_schema = StructType([
    StructField("inventory_id", StringType(), False),
    StructField("product_id", StringType(), False),
    StructField("warehouse_id", StringType(), False),
    StructField("warehouse_name", StringType(), False),
    StructField("location", StringType(), False),
    StructField("stock_quantity", IntegerType(), False),
    StructField("reserved_quantity", IntegerType(), False),
    StructField("available_quantity", IntegerType(), False),
    StructField("reorder_level", IntegerType(), False),
    StructField("last_restocked", DateType(), False),
    StructField("last_audit", DateType(), False),
    StructField("created_timestamp", TimestampType(), False)
])

print("Schema defined for inventory data")


Schema defined for inventory data


In [0]:
# Read and validate inventory data
try:
    # Read CSV files with schema validation
    df_inventory = spark.read.schema(inventory_schema).csv(source_dir, header=True, dateFormat="yyyy-MM-dd", timestampFormat="yyyy-MM-dd HH:mm:ss")
    
    # Add processing metadata
    df_inventory = df_inventory.withColumn("processed_timestamp", F.current_timestamp()) \
                              .withColumn("batch_id", F.lit(datetime.now().strftime("%Y%m%d_%H%M%S"))) \
                              .withColumn("source_system", F.lit("ecommerce_inventory"))
    
    # Data quality checks
    total_records = df_inventory.count()
    null_inventory_ids = df_inventory.filter(F.col("inventory_id").isNull()).count()
    negative_stock = df_inventory.filter(F.col("stock_quantity") < 0).count()
    negative_reserved = df_inventory.filter(F.col("reserved_quantity") < 0).count()
    invalid_available = df_inventory.filter(F.col("available_quantity") < 0).count()
    
    print(f"Total records processed: {total_records}")
    print(f"Records with null inventory_id: {null_inventory_ids}")
    print(f"Records with negative stock: {negative_stock}")
    print(f"Records with negative reserved: {negative_reserved}")
    print(f"Records with invalid available: {invalid_available}")
    
    # Filter out valid records - Fixed boolean logic
    df_valid_inventory = df_inventory.filter(
        (F.col("inventory_id").isNotNull()) & 
        (F.col("stock_quantity") >= 0) & 
        (F.col("reserved_quantity") >= 0) & 
        (F.col("available_quantity") >= 0)
    )
    
    # Capture invalid records for error handling - Fixed boolean logic
    df_invalid_inventory = df_inventory.filter(
        (F.col("inventory_id").isNull()) | 
        (F.col("stock_quantity") < 0) | 
        (F.col("reserved_quantity") < 0) | 
        (F.col("available_quantity") < 0)
    )
    
    valid_records = df_valid_inventory.count()
    invalid_records = df_invalid_inventory.count()
    
    print(f"Valid records: {valid_records}")
    print(f"Invalid records: {invalid_records}")
    
except Exception as e:
    print(f"Error reading inventory data: {str(e)}")
    raise


Total records processed: 20
Records with null inventory_id: 0
Records with negative stock: 0
Records with negative reserved: 0
Records with invalid available: 0
Valid records: 20
Invalid records: 0


In [0]:
# Data enrichment - Inventory analytics
try:
    # Calculate inventory turnover metrics
    df_valid_inventory = df_valid_inventory.withColumn(
        "stock_utilization_rate",
        F.when(F.col("stock_quantity") > 0, F.col("reserved_quantity") / F.col("stock_quantity"))
         .otherwise(F.lit(0))
    )
    
    # Create stock status categories
    df_valid_inventory = df_valid_inventory.withColumn(
        "stock_status",
        F.when(F.col("available_quantity") == 0, "Out of Stock")
         .when(F.col("available_quantity") <= F.col("reorder_level"), "Reorder Required")
         .when(F.col("available_quantity") <= F.col("reorder_level") * 2, "Low Stock")
         .otherwise("In Stock")
    )


    print("Data enrichment completed")


except Exception as e:
    print(f"Error in data enrichment: {str(e)}")
    raise

Data enrichment completed


In [0]:
# Write valid data to staging table
try:
    # Create or overwrite staging table
    df_valid_inventory.write.format("delta").mode("overwrite").saveAsTable(stage_table)
    print(f"Successfully loaded {valid_records} valid inventory records to staging table")
    
except Exception as e:
    print(f"Error writing to staging table: {str(e)}")
    raise


Successfully loaded 20 valid inventory records to staging table


In [0]:
# Archive processed files
try:
    # List all files in the source directory
    files = dbutils.fs.ls(source_dir)
    
    archived_count = 0
    for file in files:
        if file.name.endswith('.csv'):
            src_path = file.path
            archive_path = archive_dir + file.name
            
            # Move the file to archive
            dbutils.fs.mv(src_path, archive_path)
            archived_count += 1
            print(f"Archived: {file.name}")
    
    print(f"Successfully archived {archived_count} files")
    
except Exception as e:
    print(f"Error archiving files: {str(e)}")
    raise


Archived: inventory_2025_01_15.csv
Successfully archived 1 files
